In [2]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import scipy

import collections
import datetime
import time

import geopandas as gpd

import numpy as np
 
from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from sklearn import linear_model
import statsmodels.api as sm

from linearmodels import PanelOLS, FamaMacBeth
from scipy import stats

import itertools


# Direct Effects
## Grab Data

In [32]:
stockData = pd.read_csv("../../data/companyData/stocksWithControlsWeather.csv").\
    drop(columns = {'Unnamed: 0'})

stockData['indQtr']  = stockData.famafrench.astype('str') + stockData.qtr.astype('str')
stockData['yearQtr'] = stockData.year.astype('str')       + stockData.qtr.astype('str')

stockData.head()

,date,gvkey,RET,qtr,year,famafrench,ageTercile,sizeTercile,profitTercile,zipcode,temp_zipQuarterLast5,precip_zipQuarterLast5,temp_annualLast5,precip_annualLast5,indQtr,yearQtr
0,2009-01-05,1690,0.042204,1,2009,36.0,0,2.0,2.0,95014,0.0,0.0,0.0,1.0,36.01,20091
1,2009-01-05,15855,-0.033784,1,2009,42.0,0,2.0,1.0,95014,0.0,0.0,0.0,1.0,42.01,20091
2,2009-01-05,140044,0.015152,1,2009,13.0,1,1.0,0.0,95014,0.0,0.0,0.0,1.0,13.01,20091
3,2009-01-06,1690,-0.016494,1,2009,36.0,0,2.0,2.0,95014,0.0,0.0,0.0,1.0,36.01,20091
4,2009-01-06,15855,0.025175,1,2009,42.0,0,2.0,1.0,95014,0.0,0.0,0.0,1.0,42.01,20091


In [33]:
stockData.columns[0:50]

Index(['date', 'gvkey', 'RET', 'qtr', 'year', 'famafrench', 'ageTercile',
       'sizeTercile', 'profitTercile', 'zipcode', 'temp_zipQuarterLast5',
       'precip_zipQuarterLast5', 'temp_annualLast5', 'precip_annualLast5',
       'indQtr', 'yearQtr'],
      dtype='object')

# Direct Effects
Look at the effects on the suppliers when they're affected directly.

The below gives us the full, clustered standard errors.

Get the weather data to start.

In [34]:
'''stockData['heatwave_annual']          = (stockData.temp_annualLast5 == 5) * 1
stockData['extremePrecip_annual']     = (stockData.precip_annualLast5 > 0) * 1'''

stockData['heatwave_zipQuarter']      = (stockData.temp_zipQuarterLast5 == 5) * 1
stockData['extremePrecip_zipQuarter'] = (stockData.precip_zipQuarterLast5 > 0) * 1

industries = range(1,44)
indVars     = ['heatwave_zipQuarter', 'extremePrecip_zipQuarter'] 
outcomeVar = 'RET'

results = pd.DataFrame()

i = 0

#############################
for ind in industries:
    indData = stockData[stockData.famafrench == ind]
    indData = pd.get_dummies(indData, columns = ['year', 'yearQtr', 'indQtr', 'famafrench']) # , 'gvkey'
    
    firms = indData['gvkey']
    
    print('#################################################',ind,indData.shape)
    
    if indData.shape[0] > 0:

        for indVar in indVars:
            i = i + 1
            
            
            print(outcomeVar, "~", indVar)        

            col = indData.pop(indVar)
            indData.insert(0, col.name, col)

            # find: concurrent ; or lagged supplier data
            X = indData.loc[:,((indData.columns.str.contains(indVar)) | \
                                (indData.columns.str.contains('yearQtr_')))] # | 
                                # (stockData.columns.str.contains('famafrench_')))] # | 
                                # (stockData.columns.str.contains('famafrench_'))] #  | \

            print(X.columns)

            y = indData[outcomeVar]

            ######################################
            # fit the model on this subset
            '''reg = linear_model.LinearRegression()
            reg.fit(X,y)

            reg.coef_[-1]'''


            #######################################
            # try the more descriptive one
            start = time.time()

            model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)

            pvals = model.pvalues[0:len(stockData.columns[stockData.columns.str.contains(indVar)])]
            coeff =  model.params[0:len(stockData.columns[stockData.columns.str.contains(indVar)])]
            print(model.summary())

            results.loc[i,'ind']        = ind
            results.loc[i,'outcomeVar'] = outcomeVar
            results.loc[i,'weatherVar'] = indVar
            # results.loc[i,'cutoffVar']  = cutoffVar
            # results.loc[i,'firmVar']    = firmVar


            results.loc[i,'effect']     = coeff[0]

            results.loc[i,'pval']       = pvals[0]

            print(time.time() - start)
results.to_csv("../../data/stockResults_byInd.csv")

################################################# 1 (4474, 62)
RET ~ heatwave_zipQuarter
Index(['heatwave_zipQuarter', 'yearQtr_20091', 'yearQtr_20092',
       'yearQtr_20093', 'yearQtr_20094', 'yearQtr_20101', 'yearQtr_20102',
       'yearQtr_20103', 'yearQtr_20104', 'yearQtr_20111', 'yearQtr_20112',
       'yearQtr_20113', 'yearQtr_20114', 'yearQtr_20121', 'yearQtr_20122',
       'yearQtr_20123', 'yearQtr_20124', 'yearQtr_20131', 'yearQtr_20132',
       'yearQtr_20133', 'yearQtr_20134', 'yearQtr_20141', 'yearQtr_20142',
       'yearQtr_20143', 'yearQtr_20144', 'yearQtr_20151', 'yearQtr_20152',
       'yearQtr_20153', 'yearQtr_20154', 'yearQtr_20161', 'yearQtr_20162',
       'yearQtr_20163', 'yearQtr_20164', 'yearQtr_20171', 'yearQtr_20172'],
      dtype='object')
                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:           

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:01:15   Log-Likelihood:             1.3959e+05
No. Observations:               59483   AIC:                        -2.791e+05
Df Residuals:                   59438   BIC:                        -2.787e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
heatwave_zipQuarter     0.0001    

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:01:16   Log-Likelihood:                 16016.
No. Observations:                7915   AIC:                        -3.194e+04
Df Residuals:                    7870   BIC:                        -3.163e+04
Df Model:                          44                                         
Covariance Type:              cluster                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
extremePrecip_zipQuarter

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:01:17   Log-Likelihood:                 30771.
No. Observations:               14308   AIC:                        -6.145e+04
Df Residuals:                   14263   BIC:                        -6.111e+04
Df Model:                          44                                         
Covariance Type:              cluster                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
extremePrecip_zipQuarter

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:01:17   Log-Likelihood:                 14784.
No. Observations:                4875   AIC:                        -2.948e+04
Df Residuals:                    4830   BIC:                        -2.919e+04
Df Model:                          44                                         
Covariance Type:              cluster                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
extremePrecip_zipQuarter

...

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:02:04   Log-Likelihood:                 16820.
No. Observations:               10213   AIC:                        -3.355e+04
Df Residuals:                   10168   BIC:                        -3.323e+04
Df Model:                          44                                         
Covariance Type:              cluster                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
extremePrecip_zipQuarter

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:02:10   Log-Likelihood:             4.0332e+05
No. Observations:              224169   AIC:                        -8.066e+05
Df Residuals:                  224124   BIC:                        -8.061e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
extremePrecip_zipQuarter

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:02:16   Log-Likelihood:             4.7213e+05
No. Observations:              179737   AIC:                        -9.442e+05
Df Residuals:                  179692   BIC:                        -9.437e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
extremePrecip_zipQuarter

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:02:19   Log-Likelihood:             2.1178e+05
No. Observations:              111953   AIC:                        -4.235e+05
Df Residuals:                  111908   BIC:                        -4.230e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
extremePrecip_zipQuarter

ZeroDivisionError: float division by zero

# Indirect Effects
This is almost exactly the same but with supplier weather in place of the direct company information.

In [22]:
stockData = pd.read_csv("../../data/companyData/supplierStocks.csv").\
    drop(columns = {'Unnamed: 0'})

stockData['indQtr']  = stockData.famafrench.astype('str') + stockData.qtr.astype('str')
stockData['yearQtr'] = stockData.year.astype('str')       + stockData.qtr.astype('str')

stockData.head()

,year,gvkey,supplier_gvkey,salecs,totalExp,suppliers,qtr,famafrench,ageTercile,sizeTercile,profitTercile,zipcode,date,RET,supplier_famafrench,temp_zipQuarterLast5,precip_zipQuarterLast5,indQtr,yearQtr
0,2009,2136,1013.0,176.0,6625.216,36,1,32.0,0,2.0,1.0,10007,2009-01-05,-0.062356,36.0,0.0,0.0,32.01,20091
1,2009,9899,1013.0,203.0,4580.358,37,1,32.0,0,2.0,2.0,75202,2009-01-05,-0.033651,36.0,0.0,0.0,32.01,20091
2,2009,2136,1013.0,176.0,6625.216,36,1,32.0,0,2.0,1.0,10007,2009-01-06,-0.016010,36.0,0.0,0.0,32.01,20091
3,2009,9899,1013.0,203.0,4580.358,37,1,32.0,0,2.0,2.0,75202,2009-01-06,-0.004573,36.0,0.0,0.0,32.01,20091
4,2009,2136,1013.0,176.0,6625.216,36,1,32.0,0,2.0,1.0,10007,2009-01-07,0.012516,36.0,0.0,0.0,32.01,20091


In [30]:
stockData['supplier_heatwave_zipQuarter']      = (stockData.temp_zipQuarterLast5 == 5) * 1
stockData['supplier_extremePrecip_zipQuarter'] = (stockData.precip_zipQuarterLast5 > 0) * 1

industries = range(1,44)
indVars     = ['supplier_heatwave_zipQuarter', 'supplier_extremePrecip_zipQuarter'] 
outcomeVar = 'RET'

results = pd.DataFrame()

i = 0

#############################
for ind in industries:
    indData = stockData[stockData.supplier_famafrench == ind]
    indData = pd.get_dummies(indData, columns = ['year', 'yearQtr', 'indQtr', 'supplier_famafrench']) # , 'gvkey'
    
    firms = indData['gvkey']
    
    print('#################################################',ind,indData.shape)
    
    if (indData.shape[0] > 50) & (len(indData.gvkey.unique()) > 1):

        for indVar in indVars:
            i = i + 1
            print(outcomeVar, "~", indVar)        

            col = indData.pop(indVar)
            indData.insert(0, col.name, col)

            # find: concurrent ; or lagged supplier data
            X = indData.loc[:,((indData.columns.str.contains(indVar)) | \
                                (indData.columns.str.contains('yearQtr_')))] # | 
                                # (stockData.columns.str.contains('famafrench_')))] # | 
                                # (stockData.columns.str.contains('famafrench_'))] #  | \

            print(X.columns)

            y = indData[outcomeVar]

            ######################################
            # fit the model on this subset
            '''reg = linear_model.LinearRegression()
            reg.fit(X,y)

            reg.coef_[-1]'''


            #######################################
            # try the more descriptive one
            start = time.time()

            model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)

            pvals = model.pvalues[0:len(stockData.columns[stockData.columns.str.contains(indVar)])]
            coeff =  model.params[0:len(stockData.columns[stockData.columns.str.contains(indVar)])]
            print(model.summary())

            results.loc[i,'ind']        = ind
            results.loc[i,'outcomeVar'] = outcomeVar
            results.loc[i,'weatherVar'] = indVar
            # results.loc[i,'cutoffVar']  = cutoffVar
            # results.loc[i,'firmVar']    = firmVar


            results.loc[i,'effect']     = coeff[0]

            results.loc[i,'pval']       = pvals[0]

            print(time.time() - start)
results.to_csv("../../data/stockResults_bySupplierInd.csv")

################################################# 1 (3010, 81)
RET ~ supplier_heatwave_zipQuarter
Index(['supplier_heatwave_zipQuarter', 'yearQtr_20091', 'yearQtr_20092',
       'yearQtr_20093', 'yearQtr_20094', 'yearQtr_20101', 'yearQtr_20102',
       'yearQtr_20103', 'yearQtr_20104', 'yearQtr_20111', 'yearQtr_20112',
       'yearQtr_20113', 'yearQtr_20114', 'yearQtr_20121', 'yearQtr_20122',
       'yearQtr_20123', 'yearQtr_20124', 'yearQtr_20131', 'yearQtr_20132',
       'yearQtr_20133', 'yearQtr_20134', 'yearQtr_20141', 'yearQtr_20142',
       'yearQtr_20143', 'yearQtr_20144', 'yearQtr_20151', 'yearQtr_20152',
       'yearQtr_20153', 'yearQtr_20154', 'yearQtr_20161', 'yearQtr_20162',
       'yearQtr_20163', 'yearQtr_20164', 'yearQtr_20171', 'yearQtr_20172',
       'yearQtr_20173', 'yearQtr_20174', 'yearQtr_20181', 'yearQtr_20182',
       'yearQtr_20183', 'yearQtr_20184', 'yearQtr_20191', 'yearQtr_20192',
       'yearQtr_20193', 'yearQtr_20194'],
      dtype='object')
               

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        13:59:10   Log-Likelihood:                 29091.
No. Observations:               10720   AIC:                        -5.809e+04
Df Residuals:                   10675   BIC:                        -5.776e+04
Df Model:                          44                                         
Covariance Type:              cluster                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
supplier_heatwav

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        13:59:10   Log-Likelihood:                 9817.3
No. Observations:                3242   AIC:                        -1.956e+04
Df Residuals:                    3205   BIC:                        -1.934e+04
Df Model:                          36                                         
Covariance Type:              cluster                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
suppli

################################################# 13 (448684, 117)
RET ~ supplier_heatwave_zipQuarter
Index(['supplier_heatwave_zipQuarter', 'yearQtr_20091', 'yearQtr_20092',
       'yearQtr_20093', 'yearQtr_20094', 'yearQtr_20101', 'yearQtr_20102',
       'yearQtr_20103', 'yearQtr_20104', 'yearQtr_20111', 'yearQtr_20112',
       'yearQtr_20113', 'yearQtr_20114', 'yearQtr_20121', 'yearQtr_20122',
       'yearQtr_20123', 'yearQtr_20124', 'yearQtr_20131', 'yearQtr_20132',
       'yearQtr_20133', 'yearQtr_20134', 'yearQtr_20141', 'yearQtr_20142',
       'yearQtr_20143', 'yearQtr_20144', 'yearQtr_20151', 'yearQtr_20152',
       'yearQtr_20153', 'yearQtr_20154', 'yearQtr_20161', 'yearQtr_20162',
       'yearQtr_20163', 'yearQtr_20164', 'yearQtr_20171', 'yearQtr_20172',
       'yearQtr_20173', 'yearQtr_20174', 'yearQtr_20181', 'yearQtr_20182',
       'yearQtr_20183', 'yearQtr_20184', 'yearQtr_20191', 'yearQtr_20192',
       'yearQtr_20193', 'yearQtr_20194'],
      dtype='object')
           

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        13:59:24   Log-Likelihood:                 62833.
No. Observations:               24544   AIC:                        -1.256e+05
Df Residuals:                   24499   BIC:                        -1.252e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
supplier_heatwav

...

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:00:09   Log-Likelihood:                 15990.
No. Observations:                5721   AIC:                        -3.189e+04
Df Residuals:                    5676   BIC:                        -3.159e+04
Df Model:                          44                                         
Covariance Type:              cluster                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
suppli

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:00:10   Log-Likelihood:                 97614.
No. Observations:               41680   AIC:                        -1.951e+05
Df Residuals:                   41635   BIC:                        -1.948e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
suppli

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:00:13   Log-Likelihood:             1.9119e+05
No. Observations:               75397   AIC:                        -3.823e+05
Df Residuals:                   75352   BIC:                        -3.819e+05
Df Model:                          44                                         
Covariance Type:              cluster                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
suppli

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:00:27   Log-Likelihood:             1.1823e+06
No. Observations:              477590   AIC:                        -2.365e+06
Df Residuals:                  477545   BIC:                        -2.364e+06
Df Model:                          44                                         
Covariance Type:              cluster                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
suppli

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jul 2022   Prob (F-statistic):                nan
Time:                        14:00:28   Log-Likelihood:                 25437.
No. Observations:               11930   AIC:                        -5.078e+04
Df Residuals:                   11885   BIC:                        -5.045e+04
Df Model:                          44                                         
Covariance Type:              cluster                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
suppli

In [27]:
len(indData.gvkey.unique() > 1)

1